In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install python_speech_features

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the VGG19 model for grayscale spectrograms
class VGG19_Spectrogram(nn.Module):
    def __init__(self, num_classes):
        super(VGG19_Spectrogram, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        
        # Modify the input layer to accept grayscale images (1 channel)
        self.vgg19.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        
        # Modify the output layer for the number of classes
        self.vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
    
    def forward(self, x):
        x = self.vgg19(x)
        return x

# Define the transformations for the dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert images to grayscale
    transforms.Resize((224, 224)),  # Resize for VGG19
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485], std=[0.229])  # Normalize
])

# Load the dataset
directory = "/content/drive/MyDrive/Augumented Spectogram"
dataset = datasets.ImageFolder(root=directory, transform=transform)

# Split the dataset into training and testing sets
train_size = int(0.67 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Encode the labels (for later use in evaluation if necessary)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(dataset.targets)

# Define the number of classes dynamically
num_classes = len(dataset.classes)

# Create the VGG19 model
model = VGG19_Spectrogram(num_classes)

# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader.dataset):.4f}")

# Save the model after training
model_save_path = "/content/drive/MyDrive/vgg19_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Evaluate the VGG19 model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print("Accuracy:", accuracy)


In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from PIL import Image

# Define the VGG19 model architecture for grayscale spectrograms
class VGG19_Spectrogram(nn.Module):
    def __init__(self, num_classes):
        super(VGG19_Spectrogram, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        
        # Modify the input layer to accept grayscale images (1 channel)
        self.vgg19.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        
        # Modify the output layer for the number of classes
        self.vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
    
    def forward(self, x):
        x = self.vgg19(x)
        return x

# Load the pre-trained VGG19 model
num_classes = 10  # Replace with your actual number of classes
model = VGG19_Spectrogram(num_classes)

# Load the trained model weights
model_load_path = "/content/drive/MyDrive/vgg19_model.pth"
model.load_state_dict(torch.load(model_load_path))
model.eval()

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert image to grayscale
    transforms.Resize((224, 224)),  # Resize for VGG19
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485], std=[0.229])  # Normalize
])

# Load the dataset to retrieve labels
# Use the same directory structure you used during training
dataset_directory = "/content/drive/MyDrive/Augumented Spectogram"
dataset = datasets.ImageFolder(root=dataset_directory, transform=transform)

# Get the class names dynamically from the dataset
labels = dataset.classes  # Automatically retrieves class names from folder structure

# Load and transform the image
image_path = "/content/02 Am I Wrong.png"
image = Image.open(image_path)
image = transform(image)
image = image.unsqueeze(0)  # Add a batch dimension

# Move the model and image to the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
image = image.to(device)

# Make a prediction
with torch.no_grad():
    output = model(image)
    _, predicted_class = torch.max(output, 1)

# Get the predicted label from the dynamically loaded labels
predicted_label = labels[predicted_class.item()]
print(f"Predicted label: {predicted_label}")
